In [1]:
from detectron2.layers import ShapeSpec
from pixel_decoder.msdeformattn import MSDeformAttnPixelDecoder
import torch
from detectron2.config import get_cfg
from detectron2.modeling import build_backbone

In [2]:
# Load deformable pixel decoder
input_shape = {"res2": ShapeSpec(channels=256, stride=4),
               "res3": ShapeSpec(channels=512, stride=8),
               "res4": ShapeSpec(channels=1024, stride=16),
               "res5": ShapeSpec(channels=2048, stride=32)}
args = {
    'input_shape': input_shape,
    'conv_dim': 256, 
    'mask_dim': 256, 
    'norm': 'GN',
    'transformer_dropout': 0.0, 
    'transformer_nheads': 8, 
    'transformer_dim_feedforward': 1024, 
    'transformer_enc_layers': 6, 
    'transformer_in_features': ['res3', 'res4', 'res5'], 
    'common_stride': 4,
}

msd = MSDeformAttnPixelDecoder(**args)

msd_weights = torch.load('../../Mask2Former/pretrained_models/coco/MSDeformAttnPixelDecoder.pkl')
msd.load_state_dict(msd_weights)

<All keys matched successfully>

In [72]:
cfg.MODEL.WEIGHTS

'/sensei-fs/users/zhiboy/projects/Mask2Former/pretrained_models/coco/R50.pkl'

In [71]:
import os
os.path.dirname(cfg.MODEL.WEIGHTS)

'/sensei-fs/users/zhiboy/projects/Mask2Former/pretrained_models/coco'

In [52]:
# Load Detectrion2 Resnet50
cfg = get_cfg()
cfg.merge_from_file('../configs/resnet50.yaml')
backbone = build_backbone(cfg)
r50_weights = torch.load(cfg.MODEL.WEIGHTS)
backbone.load_state_dict(r50_weights)

Config '../configs/resnet50.yaml' has no VERSION. Assuming it to be compatible with latest v2.


<All keys matched successfully>

In [43]:
inp = torch.randn(4, 3, 320, 512)

In [53]:
features = backbone(inp)

In [54]:
a, b, c = msd.forward_features(features)

In [60]:
a.shape, b.shape, [s.shape for s in c]

(torch.Size([4, 256, 80, 128]),
 torch.Size([4, 256, 10, 16]),
 [torch.Size([4, 256, 10, 16]),
  torch.Size([4, 256, 20, 32]),
  torch.Size([4, 256, 40, 64])])

In [62]:
for k, v in features.items():
    print(k, v.shape)

res2 torch.Size([4, 256, 80, 128])
res3 torch.Size([4, 512, 40, 64])
res4 torch.Size([4, 1024, 20, 32])
res5 torch.Size([4, 2048, 10, 16])
